## 🍎 애플 주식 데이터 10년 분석 (2015-2025)
- 애플(Apple Inc.)의 10년간(2015-2025) 주가 데이터를 활용한 분석 프로젝트

### TASK 1: 데이터 출처 설명 / 데이터 불러오기

In [ ]:
"""
## 데이터 출처 설명

본 분석에서는 Yahoo Finance에서 제공하는 애플(AAPL) 주식 데이터를 활용함.
데이터는 Python의 yfinance 라이브러리를 기반으로 수집함.

1. **데이터 소스**: Yahoo Finance API (yfinance 라이브러리 활용)
2. **수집 기간**: 2015년 1월 1일 ~ 2025년 3월 10일
3. **수집 주기**: 일별 (거래일 기준)
4. **주요 변수**:
   - Open: 시가
   - High: 고가
   - Low: 저가
   - Close: 종가 (조정종가)
   - Volume: 거래량

추가로 비교 분석을 위해 수집할 데이터...
- 나스닥 지수(^IXIC)
- 주요 경쟁사 데이터(MSFT, GOOGL, AMZN)

yfinance 라이브러리는 주식 분할, 배당금 등이 반영된 조정 주가를 제공하므로,
장기간의 주가 추세를 분석하는 데 적합!
"""
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 6)

start_date = '2015-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

print(f"데이터 수집 기간: {start_date} ~ {end_date}")

# 애플 데이터 다운로드
apple_data = yf.download('AAPL', start=start_date, end=end_date)

# 멀티인덱스 컬럼 처리
if isinstance(apple_data.columns[0], tuple):
    apple_data.columns = [col[0] for col in apple_data.columns]

# 기본 데이터 확인
print(f"다운로드 완료: 애플 주식 데이터 {len(apple_data)}행")

### TASK 2: 기초 통계 확인

In [ ]:
# 일별 수익률 계산
apple_data['Daily_Return'] = apple_data['Close'].pct_change() * 100

# 시계열 특성 추가
apple_data['Year'] = apple_data.index.year
apple_data['Month'] = apple_data.index.month
apple_data['Quarter'] = apple_data.index.quarter
apple_data['Month_Name'] = apple_data.index.strftime('%B')

print("기본 파생변수 추가 완료")
print("추가된 변수:")
print("- Daily_Return: 일별 수익률(%)")
print("- Year: 연도")
print("- Month: 월")
print("- Quarter: 분기")
print("- Month_Name: 월 이름")

print("\n===== 애플 주식 데이터 기본 정보 =====")
print(f"분석 기간: {apple_data.index.min().strftime('%Y-%m-%d')} ~ {apple_data.index.max().strftime('%Y-%m-%d')}")
print(f"총 거래일 수: {len(apple_data)}일")
print(f"변수 목록: {', '.join(apple_data.columns)}")

missing_values = apple_data.isnull().sum()
print("\n===== 결측치 확인 =====")
print(missing_values)

print("\n===== 기초 통계량 =====")
stats_df = apple_data[['Open', 'High', 'Low', 'Close', 'Volume', 'Daily_Return']].describe()
print(stats_df)

In [ ]:
# 연도별 주가 통계 - 루프 사용
yearly_stats = pd.DataFrame()
yearly_stats['Year'] = sorted(apple_data['Year'].unique())

# 각 통계 계산
for year in yearly_stats['Year']:
    year_data = apple_data[apple_data['Year'] == year]['Close']
    yearly_stats.loc[yearly_stats['Year'] == year, 'mean'] = year_data.mean()
    yearly_stats.loc[yearly_stats['Year'] == year, 'min'] = year_data.min()
    yearly_stats.loc[yearly_stats['Year'] == year, 'max'] = year_data.max()
    yearly_stats.loc[yearly_stats['Year'] == year, 'std'] = year_data.std()

# 추가 지표 계산
yearly_stats['변동성(%)'] = yearly_stats['std'] / yearly_stats['mean'] * 100
yearly_stats['상승률(%)'] = (yearly_stats['max'] - yearly_stats['min']) / yearly_stats['min'] * 100

print("\n===== 연도별 주가 통계 =====")
print(yearly_stats)

In [ ]:
# 월별 평균 수익률 - 직접 계산
monthly_returns = pd.DataFrame()
monthly_returns['Month'] = sorted(apple_data['Month'].unique())

# 각 월의 평균 수익률 계산
for month in monthly_returns['Month']:
    month_data = apple_data[apple_data['Month'] == month]['Daily_Return']
    monthly_returns.loc[monthly_returns['Month'] == month, 'Daily_Return'] = month_data.mean()

# 월 이름 추가
month_names = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
    7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
}
monthly_returns['Month_Name'] = monthly_returns['Month'].map(month_names)

print("\n===== 월별 평균 일간 수익률 =====")
print(monthly_returns[['Month_Name', 'Daily_Return']])

### TASK 3 - 분석 목적 및 인사이트 설명
해당 데이터셋 분석의 핵심 목적은 **애플 주식 변동성 분석 기반 지능형 투자 인사이트 생성**

1. 고변동성 구간 정밀 식별 및 특성화
   - 시계열 이상치 탐지 알고리즘을 통한 비정상적 변동성 구간 자동 식별
   - 변동성 클러스터링을 통한 패턴 유형화 및 통계적 유의성 검증
   - 주가 변동성과 거래량, 시장 지표 간 상관관계 분석을 통한 예측 신호 개발

2. 뉴스 기반 변동성 인과관계 분석 시스템 구축
   - Google News RSS Feed API 연동을 통한 실시간 뉴스 데이터 수집 파이프라인 구축
   - 고변동성 구간과 연관된 뉴스 기사의 자동 매핑 및 시맨틱 분석

3. LLM 기반 투자 인사이트 생성 엔진 개발
   - 변동성 패턴, 뉴스 분석 결과를 통합한 멀티모달 데이터 처리 아키텍처 설계
   - LLM을 활용한 맞춤형 투자 인사이트 생성 시스템 개발
   - 사용자 투자 성향에 최적화된 동적 인사이트 제공 메커니즘 구현

#### 주요 인사이트

1. 장기적 성장 패턴
- 2015년 평균 주가 약 27달러에서 2025년 초 236달러까지, 약 9배 성장
- 성장 가속화 시점: 2019-2020년 사이에 주가가 급격히 상승(약 85% 증가)
- 최근 고평가: 2023-2025년 주가는 계속해서 상승하여 $250 근처까지 도달

2. 변동성 패턴
- 높은 변동성 시기: 2020년은 가장 높은 변동성(23.1%)
- 안정화 추세: 2021-2023년은 변동성이 약 10% 내외로 안정화
- 극단적 가격 변동: 일별 수익률 데이터를 보면 최대 하락폭은 -12.9%, 최대 상승폭은 12.0%로, 단기간에 큰 변동이 가능함을 보여주는 결과!